In [ ]:
import tweepy
import webbrowser
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import Cursor
from tweepy import API
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import nltk
import matplotlib.pyplot as plt
import json

with open('manager.json') as f:
    data = json.load(f)
    consumer_key = data['API_key']
    consumer_secret = data['API_secret_Key']
    Bearer_Token = data['Bearer_Token']

In [ ]:

class TwitterAccountLogin():
    api= None

    def login_user(self):
        callback_url= 'oob'
        auth= tweepy.OAuthHandler(consumer_key,consumer_secret,callback_url) # Authenticate our User
        redirect_url = auth.get_authorization_url() # Get URL to direct user
        # print(redirect_url)
        webbrowser.open(redirect_url)
        user_pint_input = input ("What's the pin value? ")
        user_pint_input
        auth.get_access_token(user_pint_input)
        self.api= tweepy.API(auth)
        me = self.api.me()
        return auth

_twitterAccountLogin = TwitterAccountLogin()
auth_login = _twitterAccountLogin.login_user()

In [ ]:
class TwitterClient():

    def __init__(self,tweeter_user=None):
        auth = auth_login
        self.authenticate = auth
        self.twitter_client = API(self.authenticate)
        self.user = tweeter_user

    def get_twetter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self,num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline,id=self.user).items(num_tweets): # Get user timeline tweets
            tweets.append(tweet)
            return tweets

    def get_friend_list(self,num_friends):
        friends =[]
        for friend in Cursor(self.twitter_client.friends,id=self.user).items(num_friends): # Get number of friends
            friends.append(friend)
            return friends

    def get_home_timeline_tweets(self,num_tweets):
        home_timeline =[]
        for home in Cursor(self.twitter_client.home_timeline,id=self.user).items(num_tweets): # Get number of home tweets
            home_timeline.append(home)
            return home_timeline

_twitterClient = TwitterClient()

In [ ]:

class TwitterListener(StreamListener):

    def __init__(self, fetch_tweets_fileName):
        self.fileName = fetch_tweets_fileName

    def on_data(self,data):
        try:
            print(data)
            with open(self.fileName,'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print(f'Error on data:{str(e)}')
            return True

    def on_error(self,status):
        if status ==420:
            print(status)
            return False

In [ ]:
class TwitterStreamer():

    def stream_tweets(self,fileName,hash_tag_list):
        auth = auth_login
         # This handles twitter auth and connect to twitter streaming API
        listener = TwitterListener(fileName) # Class instance created
        stream = Stream(auth,listener) # To authenticate user
        return stream.filter(track=hash_tag_list) # streaming what this guys tweeted
    
_twitterStreamer = TwitterStreamer()

In [ ]:
class TweetAnalyser():
    """
    Functionality for analysing tweets
    """
    api =_twitterAccountLogin.api
    def clean_text(raw_text):
            raw_text = re.sub(r'@[A-Za-z0-9]+','',raw_text)
            raw_text = re.sub(r'#','',raw_text)
            raw_text = re.sub(r'RT[\s]+','',raw_text)
            raw_text = re.sub(r'https?:\/\/\S+','',raw_text)
            raw_text = re.sub(r':','',raw_text)
            return raw_text

    def tweets_to_dataframe(self,user,count=None):
        tweets = self.api.user_timeline(screen_name=f'{user}',count=20, lang='en',tweet_mode='extended')
        df = pd.DataFrame(data=[[tweet.full_text,tweet.id,tweet.retweet_count] for tweet in tweets], columns=['Tweets','id','retweet_count'])
        df['Tweets'] = df['Tweets'].apply(TweetAnalyser.clean_text)
        return df
    
    def get_tweet_attributes(self,user,count=None): # This will display all the attributes
        tweets = self.api.user_timeline(screen_name=f'{user}',count=20)
        return dir(tweets[0])

_tweetAnalyser = TweetAnalyser()

In [ ]:
fileName= 'tweets.json'
hash_tag_list1 = ['Donald Trump','Hillary Clinton','Barack Obama']
hash_tag_list2 = ['#covid-19']

# Get stream of tweets
stream_tweets = _twitterStreamer.stream_tweets(fileName,hash_tag_list2)
print(stream_tweets)

# Prints the account holders 3x tweets
user_timeline = _twitterClient.get_user_timeline_tweets(3)
# print(user_timeline)

# Gets list of friends
list_of_friends = _twitterClient.get_friend_list(5)
# print(list_of_friends)

# Get home timeline
home_timeline = _twitterClient.get_home_timeline_tweets(5)
# print(home_timeline)

In [ ]:

api = _twitterClient.get_twetter_client_api()

df = _tweetAnalyser.tweets_to_dataframe('JonnyBones',count=70)

attri = _tweetAnalyser.get_tweet_attributes('JonnyBones',count=70)

# print(df.head(20))
# print(attri)

In [ ]:
# Subjectivity and the polarity

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

print(df)


In [ ]:

# Plot the word cloud
allword = ' '.join([twts for twts in df['Tweets']])
# wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119)

def getAnalysys(score):
        if score<0:
            return 'Negative'
        else:
            return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysys)

In [ ]:

 # print all positive tweets
print('*****Positive Tweets*****')
j = 1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Analysis'][i]=='Positive'):
            print(str(j) + ')'+ sortedDF['Tweets'][i])
            print()
            j +=1

In [ ]:

# print all negative tweets
print('*****Negative Tweets*****')
j = 1
sortedDF = df.sort_values(by=['Polarity'], ascending='False')
for i in range(0,sortedDF.shape[0]):
     if (sortedDF['Analysis'][i]=='Negative'):
            print(str(j) + ')'+ sortedDF['Tweets'][i])
            print()
            j +=1
            

In [ ]:

# Plot Polarity vs Subjectivity
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
    plt.scatter(df['Polarity'][i],df['Subjectivity'][i], color='Blue')
    plt.title('Sentiment Analysis')
    plt.xlabel('Polarity')
    plt.ylabel('Subjectivity')
    plt.show()

In [ ]:

 # Get the percentage of positive tweets
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']
percent = round((ptweets.shape[0] / df.shape[0]) * 100, 1)
print(percent)


# Get the percentage of negative tweets
ntweets = df[df.Analysis == 'Negative']
ntweets = ntweets['Tweets']
percent = round((ntweets.shape[0] / df.shape[0]) * 100, 1)
print(percent)


# Show value counts

df['Analysis'].value_counts()

# Plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show()